# Imports

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Polygon
from matplotlib import colors as mat_colors
import mpl_toolkits.axisartist as axisartist
import mpl_toolkits
from mpl_toolkits.axes_grid1 import Size, Divider
import os

# Define colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
axis_color = list(colors[7]) + [1.]
color_1 = list(colors[3]) + [1.]
color_2 = list(colors[0]) + [1.]
color_3 = list(colors[4]) + [1.]
glacier_color = glacier_color = list(colors[9]) + [.5]

# Define functions

## Performance measurements

In [ ]:
def BIAS(a1, a2):
    return (a1 - a2).mean().item()


def RMSE(a1, a2):
    return np.sqrt(((a1 - a2)**2).mean()).item()


def DIFF(a1, a2):
    return np.max(np.abs(a1 - a2)).item()

## help function for heatmap axis

In [ ]:
def setup_axes(fig, rect):
    ax = axisartist.Subplot(fig, rect)
    fig.add_subplot(ax)

    return ax

## heatmap

In [ ]:
def heatmap(datasets, # first_dataset, second_dataset,
            opti_var,
            annotation=None,
            annotation_x_position=0,
            annotation_y_position=1,
            fig=None, ax=None,
            cmap='vlag',
            cmap_levels=None,
            grid_color='grey',
            grid_linewidth=1.5,
            presentation=False,
            labels_pad=-360,
            xlim=None, # here use it do define max Diff sfc_h
            nr_of_iterations=None):
    if not ax:
        ax = plt.gca()
    
    if not fig:
        fig = plt.gcf()

    if all(dataset is None for dataset in datasets):
        raise ValueError('All datasets are None!')

    # define variables for plotting
    guess_opti_var = []
    first_guess_diff = []
    true_opti_var = []
    BIAS_opti_var = []
    RMSE_opti_var = []
    DIFF_opti_var = []
    fct_opti_var = []
    times = []
    maxiters = []
    BIAS_sfc = []
    RMSE_sfc = []
    DIFF_sfc = []
    BIAS_w = []
    RMSE_w = []
    DIFF_w = []
    BIAS_fg = []
    RMSE_fg = []
    DIFF_fg = []
    BIAS_sfc_fg = []
    RMSE_sfc_fg = []
    DIFF_sfc_fg = []
    array_length = 0
    check_first_guess = None
    check_true_opti_var = None

    # create data and label variables
    for dataset in datasets:
        # check if the current dataset contains data or if the data was not available
        if dataset is None:
            guess_opti_var.append(None)
            first_guess_diff.append(None)
            true_opti_var.append(None)
            BIAS_opti_var.append(None)
            RMSE_opti_var.append(None)
            DIFF_opti_var.append(None)
            fct_opti_var.append(None)
            times.append(None)
            maxiters.append(None)
            BIAS_sfc.append(None)
            RMSE_sfc.append(None)
            DIFF_sfc.append(None)
            BIAS_w.append(None)
            RMSE_w.append(None)
            DIFF_w.append(None)
        elif type(dataset) != xr.core.dataset.Dataset:  # if no minimisation possible
            guess_opti_var.append('no_minimisation')
            first_guess_diff.append(None)
            true_opti_var.append(None)
            BIAS_opti_var.append(None)
            RMSE_opti_var.append(None)
            DIFF_opti_var.append(None)
            fct_opti_var.append(None)
            times.append(None)
            maxiters.append(None)
            BIAS_sfc.append(None)
            RMSE_sfc.append(None)
            DIFF_sfc.append(None)
            BIAS_w.append(None)
            RMSE_w.append(None)
            DIFF_w.append(None)
        else:
            # find index corresponding to max time
            max_index = len(dataset['computing_time'].values) - 1

            if nr_of_iterations is not None:
                if max_index >= nr_of_iterations:
                    max_index = nr_of_iterations - 1
            elif xlim is not None:
                # calculate all max diff surface_h
                all_DIFF_sfc_h = np.array(
                    [DIFF(dataset.true_surface_h.data,
                          dataset.surface_h.data[i-1])
                     for i in dataset.coords['nr_of_iteration'].data])
                # only consider as many points until max DIFF is smaller xlim
                if all_DIFF_sfc_h[-1] < xlim:
                    max_index = np.argmax(all_DIFF_sfc_h < xlim)

            if opti_var == 'bed_h':
                guess_opti_var.append((dataset.guessed_bed_h[max_index] - dataset.true_bed_h).values)
                first_guess_diff.append((dataset.first_guessed_bed_h - dataset.true_bed_h).values)
                true_opti_var.append(dataset.true_bed_h.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                RMSE_opti_var.append(RMSE(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                DIFF_opti_var.append(DIFF(dataset.guessed_bed_h[max_index], dataset.true_bed_h))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_bed_h, dataset.true_bed_h)
                    RMSE_fg = RMSE(dataset.first_guessed_bed_h, dataset.true_bed_h)
                    DIFF_fg = DIFF(dataset.first_guessed_bed_h, dataset.true_bed_h)
            elif opti_var == 'bed_shape':
                guess_opti_var.append((dataset.guessed_bed_shape[-1] - dataset.true_bed_shape).values)
                first_guess_diff.append((dataset.first_guessed_bed_shape - dataset.true_bed_shape).values)
                true_opti_var.append(dataset.true_bed_shape.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                RMSE_opti_var.append(RMSE(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                DIFF_opti_var.append(DIFF(dataset.guessed_bed_shape[max_index], dataset.true_bed_shape))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
                    RMSE_fg = RMSE(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
                    DIFF_fg = DIFF(dataset.first_guessed_bed_shape, dataset.true_bed_shape)
            elif opti_var == 'w0':
                guess_opti_var.append((dataset.guessed_w0[-1] - dataset.true_w0).values)
                first_guess_diff.append((dataset.first_guessed_w0 - dataset.true_w0).values)
                true_opti_var.append(dataset.true_w0.values)
                BIAS_opti_var.append(BIAS(dataset.guessed_w0[max_index], dataset.true_w0))
                RMSE_opti_var.append(RMSE(dataset.guessed_w0[max_index], dataset.true_w0))
                DIFF_opti_var.append(DIFF(dataset.guessed_w0[max_index], dataset.true_w0))
                if check_first_guess is None:
                    BIAS_fg = BIAS(dataset.first_guessed_w0, dataset.true_w0)
                    RMSE_fg = RMSE(dataset.first_guessed_w0, dataset.true_w0)
                    DIFF_fg = DIFF(dataset.first_guessed_w0, dataset.true_w0)
            else:
                raise ValueError('Unknown opti var!')
                
            fct_opti_var.append(dataset.function_calls[max_index].values)
            times.append(dataset.computing_time[max_index].values)
            maxiters.append(dataset.attrs['maxiter_reached'])
            BIAS_sfc.append(BIAS(dataset.surface_h[max_index], dataset.true_surface_h))
            RMSE_sfc.append(RMSE(dataset.surface_h[max_index], dataset.true_surface_h))
            DIFF_sfc.append(DIFF(dataset.surface_h[max_index], dataset.true_surface_h))
            BIAS_w.append(BIAS(dataset.widths[max_index], dataset.true_widths))
            RMSE_w.append(RMSE(dataset.widths[max_index], dataset.true_widths))
            DIFF_w.append(DIFF(dataset.widths[max_index], dataset.true_widths))

            # determine array length for empty lines
            if array_length == 0:
                array_length = dataset.points_with_ice[-1].values + 1
            # check that the arrays have the same number of points with ice
            elif array_length != dataset.points_with_ice[-1].values + 1:
                raise ValueError('Not the same lentgth of points with ice!!!')

            # check if all experiments start with the same true values and first guess
            # in the first round save values
            if check_first_guess is None:
                check_first_guess = first_guess_diff[-1]
                check_true_opti_var = true_opti_var[-1]
                
                # not implemented yet
                BIAS_sfc_fg = BIAS(dataset.first_guess_surface_h, dataset.true_surface_h)
                RMSE_sfc_fg = RMSE(dataset.first_guess_surface_h, dataset.true_surface_h)
                DIFF_sfc_fg = DIFF(dataset.first_guess_surface_h, dataset.true_surface_h)
                BIAS_w_fg = BIAS(dataset.first_guess_widths, dataset.true_widths)
                RMSE_w_fg = RMSE(dataset.first_guess_widths, dataset.true_widths)
                DIFF_w_fg = DIFF(dataset.first_guess_widths, dataset.true_widths)

            # after first round compare all values to first ones to make sure comparing the same start conditions
            else:
                if np.sum(check_true_opti_var - true_opti_var[-1]) != 0:
                    raise ValueError('Not the same true control variable!!!')
                if ~np.isclose(np.sum(check_first_guess - first_guess_diff[-1]), 0):
                    raise ValueError('Not the same first guess!!!')

    # create variables for ploting (data and y label)
    data = []
    y_labels = []

    # first add heading
    data.append(np.empty((array_length)) * np.nan)
    if not presentation:
        if opti_var == 'bed_h':
            y_labels.append(r'    RMSE_b,  DIFF_b,  RMSE_s,  DIFF_s,  fct,  $T_{cpu}$')
        elif opti_var == 'bed_shape':
            y_labels.append(r'   RMSE_Ps, DIFF_Ps,  RMSE_w,  DIFF_w,  fct,  $T_{cpu}$')
        elif opti_var == 'w0':
            y_labels.append(r'   RMSE_w0, DIFF_w0,  RMSE_w,  DIFF_w,  fct,  $T_{cpu}$')
        else:
            raise ValueError('Unknown opti_var !')
        y_label_variable_format = '{:7.2f}, {: 7.2f}, {:7.2f}, {:7.2f}'
    else:
        if opti_var == 'bed_h':
            y_labels.append('   DIFF_b, DIFF_s, fct, $T_{cpu}$')
        elif opti_var == 'bed_shape':
            y_labels.append('  DIFF_Ps, DIFF_w, fct, $T_{cpu}$')
        elif opti_var == 'w0':
            y_labels.append('  DIFF_w0, DIFF_w, fct, $T_{cpu}$')
        else:
            raise ValueError('Unknown opti_var !')
        y_label_variable_format = '{: 6.2f}, {:6.2f}'

    if not presentation:
        # add first guess
        data.append(check_first_guess)
        if opti_var == 'bed_h':
            y_labels.append(('fg:' + y_label_variable_format).format(RMSE_fg, DIFF_fg,
                                                                     RMSE_sfc_fg, DIFF_sfc_fg))
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(('fg:' + y_label_variable_format).format(RMSE_fg, DIFF_fg,
                                                                     RMSE_w_fg, DIFF_w_fg))
        else:
            raise ValueError('Unknown opti_var !')
    else:
        # add first guess
        data.append(check_first_guess)
        if opti_var == 'bed_h':
            y_labels.append(('fg:' + y_label_variable_format).format(DIFF_fg, DIFF_sfc_fg))
        elif opti_var in ['bed_shape', 'w0']:
            y_labels.append(('fg:' + y_label_variable_format).format(DIFF_fg, DIFF_w_fg))
        else:
            raise ValueError('Unknown opti_var !')
    
    # add two format placeholders for fct_calls and time
    y_label_variable_format += ', {:3d}, {:3.0f}s'

    # add all other data with empty line for None datasets
    for i, guess in enumerate(guess_opti_var):
        if guess is None:
            data.append(np.empty((array_length)) * np.nan)
            if i < 9:
                y_labels.append((' ' + chr(65 + i) + ':    NO DATAFILE FOUND'))
            else:
                y_labels.append((' ' + chr(65 + i) + ':    NO DATAFILE FOUND'))
        elif type(guess) is str:
            data.append(np.empty((array_length)) * np.nan)
            if i < 9:
                y_labels.append((' ' + chr(65 + i) + ':    NO Minimisation Possible'))
            else:
                y_labels.append((' ' + chr(65 + i) + ':    NO Minimisation Possible'))
        else:
            data.append(guess)
            if i < 9:
                y_label_text = (' ' + chr(65 + i) + ':' + y_label_variable_format)
            else:
                y_label_text = (' ' + chr(65 + i) + ':' + y_label_variable_format)
            
            #if maxiters[i] == 'yes':
            #    y_label_text += '+'

            if opti_var == 'bed_h':
                if not presentation:
                    y_labels.append(y_label_text.format(RMSE_opti_var[i],
                                                        DIFF_opti_var[i],
                                                        RMSE_sfc[i],
                                                        DIFF_sfc[i],
                                                        fct_opti_var[i],
                                                        times[i]))
                else:
                    y_labels.append(y_label_text.format(DIFF_opti_var[i],
                                                        DIFF_sfc[i],
                                                        fct_opti_var[i],
                                                        times[i]))
            elif opti_var in ['bed_shape', 'w0']:
                if not presentation:
                    y_labels.append(y_label_text.format(RMSE_opti_var[i],
                                                        DIFF_opti_var[i],
                                                        RMSE_w[i],
                                                        DIFF_w[i],
                                                        fct_opti_var[i],
                                                        times[i]))
                else:
                    y_labels.append(y_label_text.format(DIFF_opti_var[i],
                                                        DIFF_w[i],
                                                        fct_opti_var[i],
                                                        times[i]))
            else:
                raise ValueError('Unknown opti_var !')

    # make data an numpy array
    data = np.array(data)

    #choose colormap
    if not cmap_levels:
        color_nr = 100
        if opti_var == 'bed_h':
            #cmap_limit = np.max(np.abs(check_first_guess))
            cmap_limit = np.max(np.array([np.abs(np.floor(np.nanmin(np.array(data)))),
                                          np.abs(np.ceil(np.nanmax(np.array(data))))]))
        elif opti_var in ['bed_shape', 'w0']:
            cmap_limit = np.max(np.abs(check_first_guess))
            #cmap_limit = np.max(np.array([np.abs(np.floor(np.nanmin(np.array(data)) * 10)),
            #                              np.abs(np.ceil(np.nanmax(np.array(data)) * 10))])) / 10
        else:
            raise ValueError('Unknown opti var!!')
        #if (np.min(data) < 0) & (np.max(data) > 0):
        cmap_levels = np.linspace(-cmap_limit, cmap_limit, color_nr, endpoint=True)
        #elif (np.min(data) < 0) & (np.max(data) =< 0):
        #    cmap_levels = np.linspace(-cmap_limit, 0, color_nr, endpoint=True)
        #elif (np.min(data) >= 0) & (np.max(data) > 0)
    else:
        color_nr = len(cmap_levels) - 1
    
    rel_color_steps = np.arange(color_nr)/color_nr
    if cmap == 'rainbow':
        colors = cm.rainbow(rel_color_steps)
    elif cmap == 'vlag':
        colors = sns.color_palette('vlag', color_nr)
    elif cmap == 'icefire':
        colors = sns.color_palette('icefire', color_nr)
    elif cmap == 'Spectral':
        colors = sns.color_palette('Spectral_r', color_nr)
    
    cmap = LinearSegmentedColormap.from_list('custom', colors, N=color_nr)
    cmap.set_bad(color='white')
    norm = mat_colors.BoundaryNorm(cmap_levels, cmap.N)

    # plot heatmap
    im = plt.imshow(data, aspect='auto', interpolation=None, cmap=cmap, norm=norm, alpha=1.)
    
    # Turn spines and ticks off and create white frame.
    for key, spine in ax.axis.items():
        spine.major_ticks.set_visible(False)
        spine.minor_ticks.set_visible(False)
        spine.line.set_visible(False)
        # spine.line.set_color(grid_color)
        # spine.line.set_linewidth(0) #grid_linewidth)
    
    # set y ticks
    ax.set_yticks(np.arange(data.shape[0]))
                
    ax.set_yticklabels(y_labels)
    #for tick in ax.get_yticklabels():
    #    tick.set_fontname("Arial")

    # align yticklabels left
    ax.axis["left"].major_ticklabels.set_ha("left")
    
    # set pad to put labels over heatmap
    ax.axis["left"].major_ticklabels.set_pad(labels_pad)

    # set y minor grid
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", axis='y', color=grid_color, linestyle='-', linewidth=grid_linewidth)
    
    # set x ticklabels off
    ax.set_xticklabels([])
    
    # create colorbar
    cax = ax.inset_axes([1.01, 0.1, 0.03, 0.8]) 
    #cax = fig.add_axes([0.5, 0, 0.01, 1])
    cbar = fig.colorbar(im, cax=cax, boundaries=cmap_levels, spacing='proportional',)
    cbar.set_ticks([np.min(cmap_levels),0,np.max(cmap_levels)])
    if opti_var == 'bed_h':
        cbar.set_ticklabels(['{:d}'.format(int(-cmap_limit)), '0' ,'{:d}'.format(int(cmap_limit))])
    elif opti_var == 'bed_shape':
        cbar.set_ticklabels(['{:.1f}'.format(-cmap_limit), '0' ,'{:.1f}'.format(cmap_limit)])
    elif opti_var == 'w0':
        cbar.set_ticklabels(['{:d}'.format(int(-cmap_limit)), '0' ,'{:d}'.format(int(cmap_limit))])
    else:
        raise ValueError('Unknown opti var!!')
    #cbar.ax.set_ylabel(cbarlabel,)
    
    # set title
    #ax.set_title(title)
    
    if annotation is not None:
        # include text
        ax.text(annotation_x_position, annotation_y_position, 
                 annotation,
                 horizontalalignment='left',
                 verticalalignment='center',
                 transform=ax.transAxes)
    
    return im

## performance plot

In [ ]:
def performance_plot_one_y_axis(ax,
                     datasets,
                     fig=None,
                     # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                     # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                     # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                     # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                     # 'widths RMSE', 'widths Diff', 'widths Bias'
                     performance_measurement='bed_h RMSE',
                     colors=[color_1],
                     xlim=None,
                     y_label='',
                     annotation=None,
                     annotation_x_position=-0.2,
                     annotation_y_position=1,
                     lw=2,
                     fontsize=25,
                     ms=10,
                     nr_of_iterations=None
                    ):
    if not fig:
        fig = plt.gcf()

    measure = performance_measurement
    line_styles = []
    dataset = datasets[0]
    #colors = [color_1, color_2]
    
    
    for measure in performance_measurement:
        for color, dataset in zip(colors, datasets):
            max_index = len(dataset['computing_time'].values) - 1
            if nr_of_iterations is not None:
                max_index = nr_of_iterations - 1
            elif xlim is not None:
                # only consider as many points until max of xlim is reached
                if dataset['computing_time'].values[-1] > xlim[1]:
                    max_index = np.argmax(dataset['computing_time'].values > xlim[1])
            all_x = []
            all_y = []
            # include time 0 for first guess
            tmp_x = [0]

            # add first guess values 
            if measure == 'bed_h RMSE':
                line_styles.append('.-')
                tmp_y = [RMSE(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]
            elif measure == 'bed_h Diff':
                tmp_y = [DIFF(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]
            elif measure == 'bed_h Bias':
                tmp_y = [BIAS(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]

            elif measure == 'bed_shape RMSE':
                line_styles.append('.:')
                tmp_y = [RMSE(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]
            elif measure == 'bed_shape Diff':
                tmp_y = [DIFF(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]
            elif measure == 'bed_shape Bias':
                tmp_y = [BIAS(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]

            elif measure == 'w0 RMSE':
                line_styles.append('.:')
                tmp_y = [RMSE(dataset['first_guessed_w0'], dataset['true_w0'])]
            elif measure == 'w0 Diff':
                tmp_y = [DIFF(dataset['first_guessed_w0'], dataset['true_w0'])]
            elif measure == 'w0 Bias':
                tmp_y = [BIAS(dataset['first_guessed_w0'], dataset['true_w0'])]

            elif measure == 'sfc_h RMSE':
                line_styles.append('.--')
                tmp_y = [RMSE(dataset['first_guess_surface_h'], dataset['true_surface_h'])]
            elif measure == 'sfc_h Diff':
                tmp_y = [DIFF(dataset['first_guess_surface_h'], dataset['true_surface_h'])]
            elif measure == 'sfc_h Bias':
                tmp_y = [DIFF(dataset['first_guess_surface_h'], dataset['true_surface_h'])]

            elif measure == 'widths RMSE':
                tmp_y = [RMSE(dataset['first_guess_widths'], dataset['true_widths'])]
            elif measure == 'widths Diff':
                tmp_y = [DIFF(dataset['first_guess_widths'], dataset['true_widths'])]
            elif measure == 'widths Bias':
                tmp_y = [DIFF(dataset['first_guess_widths'], dataset['true_widths'])]

            else:
                raise ValueError('Unknown performance measurement!')

            for i in dataset.coords['nr_of_iteration'].values[:max_index + 1] - 1:
                tmp_x.append(dataset['computing_time'][i].data)
                if measure == 'bed_h RMSE':
                    tmp_y.append(RMSE(dataset['guessed_bed_h'][i], dataset['true_bed_h']))
                elif measure == 'bed_h Diff':
                    tmp_y.append(DIFF(dataset['guessed_bed_h'][i], dataset['true_bed_h']))
                elif measure == 'bed_h Bias':
                    tmp_y.append(BIAS(dataset['guessed_bed_h'][i], dataset['true_bed_h']))

                elif measure == 'bed_shape RMSE':
                    tmp_y.append(RMSE(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))
                elif measure == 'bed_shape Diff':
                    tmp_y.append(DIFF(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))
                elif measure == 'bed_shape Bias':
                    tmp_y.append(BIAS(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))

                elif measure == 'w0 RMSE':
                    tmp_y.append(RMSE(dataset['guessed_w0'][i], dataset['true_w0']))
                elif measure == 'w0 Diff':
                    tmp_y.append(DIFF(dataset['guessed_w0'][i], dataset['true_w0']))
                elif measure == 'w0 Bias':
                    tmp_y.append(BIAS(dataset['guessed_w0'][i], dataset['true_w0']))

                elif measure == 'sfc_h RMSE':
                    tmp_y.append(RMSE(dataset['surface_h'][i], dataset['true_surface_h']))
                elif measure == 'sfc_h Diff':
                    tmp_y.append(DIFF(dataset['surface_h'][i], dataset['true_surface_h']))
                elif measure == 'sfc_h Bias':
                    tmp_y.append(BIAS(dataset['surface_h'][i], dataset['true_surface_h']))

                elif measure == 'widths RMSE':
                    tmp_y.append(RMSE(dataset['widths'][i], dataset['true_widths']))
                elif measure == 'widths Diff':
                    tmp_y.append(DIFF(dataset['widths'][i], dataset['true_widths']))
                elif measure == 'widths Bias':
                    tmp_y.append(BIAS(dataset['widths'][i], dataset['true_widths']))

                else:
                    raise ValueError('Unknown performance measurement!')

            all_x.append(tmp_x)
            all_y.append(tmp_y)

            for i, (x, y) in enumerate(zip(all_x, all_y)):
                ax.plot(x, y,
                        line_styles[-1],
                        lw=lw,
                        ms=ms,
                        c=color)

        #ax.legend((),(),title=measure, loc='best')
        #ax.axvline(60, alpha=0.5, c='gray', ls='--')
        # ax.axvline(20, alpha=0.5, c='gray', ls='--')
        #if xlim is not None:
        #    ax.set_xlim(xlim)
    ax.tick_params(axis='both', colors=axis_color, width=lw)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_color(axis_color)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.set_xlabel(r'$T_{cpu}$', fontsize=fontsize, c=axis_color)
    ax.set_ylabel(y_label, fontsize=fontsize, c=axis_color)
    
    if annotation is not None:
        ax.text(annotation_x_position, annotation_y_position,
                annotation,
                horizontalalignment='left',
                verticalalignment='center',
                transform = ax.transAxes)

## convert iterations to datasets

In [ ]:
def convert_iterations_to_dataset(dataset, timesteps=1):
    #total_iterations = dataset.coords['nr_of_iteration'].values
    #relative_iteration = int(total_iterations[-1] / (timesteps - 1))
    #plot_iterations = total_iterations[::relative_iteration]
    #if len(plot_iterations) == timesteps - 1:
    #    plot_iterations= np.append(plot_iterations, total_iterations[-1])
    #elif len(plot_iterations) == timesteps:
    #    plot_iterations[-1] = total_iterations[-1]
    #else:
    #    raise ValueError('Something went wrong with calculation of timesteps of iterations!')
    plot_iterations = dataset.coords['nr_of_iteration'].values

    timeseries_datasets = []
    for iteration in plot_iterations - 1:
        tmp_dataset = xr.Dataset(
                    data_vars={
                        'true_bed_h':
                            (['points_with_ice'],
                             dataset.true_bed_h),
                        'first_guessed_bed_h':
                            (['points_with_ice'],
                             dataset.first_guessed_bed_h),
                        'guessed_bed_h':
                            (['nr_of_iteration', 'points_with_ice'],
                             np.array([dataset.guessed_bed_h[iteration].values])),
                        'true_bed_shape':
                            (['points_with_ice'],
                             dataset.true_bed_shape),
                        'first_guessed_bed_shape':
                            (['points_with_ice'],
                             dataset.first_guessed_bed_shape),
                        'guessed_bed_shape':
                            (['nr_of_iteration', 'points_with_ice'],
                             np.array([dataset.guessed_bed_shape[iteration].values])),
                        'function_calls':
                            (['nr_of_iteration'],
                             np.array([dataset.function_calls[iteration].values])),
                        'true_surface_h':
                            (['total_distance'],
                             dataset.true_surface_h),
                        'first_guess_surface_h':
                            (['total_distance'],
                             dataset.first_guess_surface_h),
                        'surface_h':
                            (['nr_of_iteration', 'total_distance'],
                             np.array([dataset.surface_h[iteration].values])),
                        'true_widths':
                            (['total_distance'],
                             dataset.true_widths),
                        'first_guess_widths':
                            (['total_distance'],
                             dataset.first_guess_widths),
                        'widths':
                            (['nr_of_iteration', 'total_distance'],
                             np.array([dataset.widths[iteration].values])),
                        'computing_time':
                            (['nr_of_iteration'],
                             np.array([dataset.computing_time[iteration].values]))
                    },
                    coords={
                        'total_distance': dataset.coords['total_distance'],
                        'points_with_ice': dataset.coords['points_with_ice'],
                        'nr_of_iteration': ([1])
                    },
                    attrs={
                        #'computing_time': -1,
                        'maxiter_reached': False
                    }
        )
        #if iteration == plot_iterations[-1] - 1:
        #    tmp_dataset.attrs['computing_time'] = dataset.attrs['computing_time']
        timeseries_datasets.append(tmp_dataset)
    return timeseries_datasets

## help function to put only text in axis

In [ ]:
def set_text(ax, text, alignment='left', facecolor=[0.5,0.5,0.5], fontsize=12):
    ax.text(0, 0, text,
            fontsize=fontsize,
            verticalalignment='center', horizontalalignment=alignment)

    ax.set_ylim([-1, 1])

    if alignment == 'left':
        ax.set_xlim([0, 1])
    elif alignment == 'center':
        ax.set_xlim([-1, 1])

    if type(ax.axis) == mpl_toolkits.axes_grid1.mpl_axes.Axes.AxisDict:
        for key, spine in ax.axis.items():
                spine.major_ticks.set_visible(False)
                spine.minor_ticks.set_visible(False)
                spine.line.set_visible(False)
    else:
        ax.axis('off')

    ax.set_yticklabels([])
    ax.set_xticklabels([])

    ax.set_facecolor(facecolor)

## legend plot

In [ ]:
def add_legend(ax,
               title,
               fontsize,
               lw,
               ms,
               ncol,
               labels,
               colors,
               line_styles):
    
    for label, color, line_style in zip(labels, colors, line_styles):
        ax.plot([],
                [],
                line_style,
                lw=lw,
                ms=ms,
                c=color,
                label=label)
    
    l = ax.legend(loc='center',
              fontsize=fontsize,
              title=title,
              ncol=ncol)
    plt.setp(l.get_title(), multialignment='center')
    ax.axis('off')

# rectangular figure

## Define function

In [ ]:
def plot_rec_overview(input_folder,
                         output_folder,
                         filename,
                         bed_geometry,
                         control_var,
                         reg_parameter,
                         suffix,
                         glacier_state,
                         performance_measures,
                         xlim=[0,60],
                         ylims=[[0,16],[0,10]],
                         lw = 3.,
                         ms = 15.,
                         labels_pad=-430,
                         colors=['tab:blue', 'tab:red'],
                         file_format='pdf',
                         dpi=300,
                         fontsize=25,
                         show_title = False,
                         line_height_multiplier=1.,
                         facecolor = [0.9, 0.9, 0.9],
                         nr_of_iterations=None,
                         spinup_years=None,
                         save_figure=False,
                         legend_title=r'Experiments with $\lambda_0$ = 1 and $\lambda_1$ = 10',
                         extra_annotation=None):
    plt.rcParams['font.family'] = 'monospace'
    mpl.rcParams.update({'font.size': fontsize})

    fig = plt.figure(figsize=(1, 1), facecolor=facecolor)

    # define grid for axis

    #        heading | legned
    #------------------------
    # identifier | spatial subplots | performance subplots

    # multiplier for variable plot height
    if len(input_folder) == 1:
        height_multiplier = 1 #len(suffix)
    else:
        height_multiplier = 1 # len(input_folder)

    # define fixed size of subplot
    subplot_width = 6
    subplot_height = .4 + line_height_multiplier * height_multiplier
    subplot_separation_x = 2.5
    subplot_separation_y = 0.5

    # define fixed x size for identifier (first columns)
    identifier_width = 1.5
    identifier_separation = .1

    # define separation identifier to subplots
    separation_identifier_plots = .5

    # define hight of header line
    height_header = 1.5

    #define separation identifier/subplots to heading
    separation_heading_plots = .8

    #define separation heading to title
    separation_heading_title = .1

    # define hight of title 
    height_title = .5

    # fixed size in inch
    # along x axis                                                              x-index for locator
    horiz = [Size.Fixed(identifier_width),                                    # 0 first identifier
             Size.Fixed(identifier_separation),   
             Size.Fixed(identifier_width),                                    # 2 second identifier
             Size.Fixed(separation_identifier_plots),
             Size.Fixed(subplot_width),                                       # 4 spatial subplot column
             Size.Fixed(subplot_separation_x),
             Size.Fixed(subplot_width),                                       # 6 performance subplot column
            ]
                                                                                  # y-index for locator
    vert = [Size.Fixed(subplot_height),                                       # 0 6th row subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 2 5th row subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 4 4th row subplot 
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 6 3rd row subplot 
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 8 2nd row subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 10 1st row subplot 
            Size.Fixed(separation_heading_plots),
            Size.Fixed(height_header),                                        # 12 header | legend 
           ]

    if show_title:
        vert += [Size.Fixed(separation_heading_title),
                 Size.Fixed(height_title)                                          # 14 title
                ]

    if extra_annotation is not None:
        vert += [Size.Fixed(0.5)]                                             # 13 extra annotation

    rect = (0., 0., 1., 1.)  # Position of the grid in the figure

    # divide the axes rectangle into grid whose size is specified by horiz * vert
    divider = Divider(fig, rect, horiz, vert, aspect=False)

    # include header line
    if bed_geometry == 'rec':
        shape_header = 'widhts'
        title_geometry = 'Rectangular'
    elif bed_geometry == 'par':
        shape_header = 'shape factor'
        title_geometry = 'Parabolic'
    elif bed_geometry == 'tra':
        shape_header == 'w0'
        title_geometry = 'Trapezoidal'
    else:
        raise ValueError('Unknown bed geometry!')

    if glacier_state == 'equ':
        title_glacier_state = 'equilibrium'
    elif glacier_state == 'adv':
        title_glacier_state = 'advancing'
    elif glacier_state == 'ret':
        title_glacier_state = 'retreating'
    elif glacier_state == 'ret_spinup':
        title_glacier_state = 'retreating with spinup'
    else:
        raise ValueError('Unknown glacier state')

    if show_title:
        # Include title
        ax = fig.subplots()
        set_text(ax, title_geometry + ' bed shape, '
                 + title_glacier_state + ' Experiments, reg_parameters = '
                 + reg_parameter,
                 facecolor=facecolor,
                 alignment='center',
                 fontsize=fontsize)
        ax.set_axes_locator(divider.new_locator(nx=0, nx1=7, ny=14))
        
    if extra_annotation is not None:
        # Include extra_annotation
        ax = fig.subplots()
        set_text(ax, extra_annotation,
                 facecolor=facecolor,
                 alignment='left',
                 fontsize=fontsize)
        ax.set_axes_locator(divider.new_locator(nx=0, ny=13))

    # include header
    for i, item in enumerate(['bed',
                              shape_header]):
        ax = setup_axes(fig, 111)
        set_text(ax, item, facecolor=facecolor, alignment='center', fontsize=fontsize)
        ax.set_axes_locator(divider.new_locator(nx=i*2, ny=12))

    # include legend of performance plot
    ax = fig.subplots()
    add_legend(ax,
               title=legend_title,
               fontsize=fontsize,
               lw=lw,
               ms=ms,
               ncol=2,
               labels=[r'fg: first guess $b$ difference',
                       r' A: $b$ difference after ' + str(nr_of_iterations) + ' Iterations',
                       r'RMSE of $b$ and $b_{t}$',
                       r'RMSE of $s^{e}_{m}$ and $s^{e}_{o}$'],
               colors=['none', 'none', color_1, color_1],
               line_styles=['', '', '.-', '.--'])
    ax.set_axes_locator(divider.new_locator(nx=4, nx1=7, ny=12))

    # include subplots
    bed_heights = {'line': 'linear', 'clif': 'cliff', 'rand': 'random'}
    bed_shapes = {'cons': 'constant', 'wide':'wide top'}  # 'rand': 'rand'}

    # variables to keep track of current row
    row_nr = 10

    # variable for creation of filename
    filename_bed_geometry = (bed_geometry + '_')

    # define colormap
    if control_var == 'bed_h':
        cmap='Spectral'
    else:
        cmap='vlag'

    # add subplots
    for bed_height in bed_heights:

        # add bed height type to filename
        filename_bed_height = (filename_bed_geometry + bed_height + '_')

        for bed_shape in bed_shapes:
            # set first identifier (for two lines)
            ax = setup_axes(fig, 111)
            set_text(ax, bed_heights[bed_height], facecolor=facecolor, alignment='center', fontsize=fontsize)
            ax.set_axes_locator(divider.new_locator(nx=0, ny=row_nr))
            # set second identifier
            ax = setup_axes(fig, 111)
            set_text(ax, bed_shapes[bed_shape], facecolor=facecolor, alignment='center', fontsize=fontsize)
            ax.set_axes_locator(divider.new_locator(nx=2, ny=row_nr))

            # add bed shape type to filename
            filename_bed_shape = (filename_bed_height + bed_shape + '_')

            # add glacier state and control var
            filename_raw = (filename_bed_shape + glacier_state + '_' + control_var)

            if spinup_years is None:
                # add suffix and ending to filename
                filename_nc = filename_raw + suffix + '.nc'
            else:
                filename_nc = filename_raw + '_' + str(spinup_years) + suffix + '.nc'

            # get datset
            if filename_nc in os.listdir(input_folder):
                with xr.open_dataset(input_folder + filename_nc) as ds:
                    dataset = ds
            else:
                dataset = None

            # plot spatial distribution
            ax = setup_axes(fig, 111)
            im = heatmap([dataset],
                         opti_var=control_var,
                         fig=fig,
                         ax=ax,
                         cmap=cmap,
                         grid_color=facecolor,
                         presentation=True,
                         labels_pad=labels_pad,
                         xlim=xlim,
                         nr_of_iterations=nr_of_iterations)
            ax.set_axes_locator(divider.new_locator(nx=4, ny=row_nr))

            # plot performance plot
            ax = fig.subplots()
            ax.set_axes_locator(divider.new_locator(nx=6, ny=row_nr))
            #ax2 = ax.twinx()
            #ax2.set_axes_locator(divider.new_locator(nx=6, ny=row_nr))
            performance_plot_one_y_axis(ax,
                     [dataset],
                     fig=fig,
                     # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                     # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                     # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                     # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                     # 'widths RMSE', 'widths Diff', 'widths Bias'
                     performance_measurement=performance_measures,
                     xlim=xlim,
                     y_label='m',
                     annotation=None,
                     annotation_x_position=-0.2,
                     annotation_y_position=1,
                     lw=lw,
                     fontsize=fontsize,
                     ms=ms,
                     nr_of_iterations=nr_of_iterations
                    )


            row_nr -= 2

    fig.savefig(output_folder + filename + '.' + file_format,format=file_format,bbox_inches='tight',dpi=dpi)

## plot equilibrium figure from inital ice surface

In [ ]:
input_folder = 'plot_data/rectangular_experiments/'

plot_rec_overview(input_folder=input_folder,
                 output_folder='',
                 filename='rec_equilibrium_inital_sfc_overview',
                 bed_geometry='rec',
                 control_var='bed_h',
                 reg_parameter='[1., 1., 1., 10.]',
                 suffix='reg10',
                 glacier_state='equ',
                 performance_measures=['bed_h RMSE', 'sfc_h RMSE'],
                 xlim=None,
                 ylims=[[0,16],[0,10]],
                 lw = 3.,
                 ms = 15.,
                 labels_pad=-430,
                 colors=[color_1, color_2],
                 file_format='pdf',
                 dpi=300,
                 fontsize=25,
                 show_title = False,
                 line_height_multiplier=1.,
                 facecolor = 'white',
                 nr_of_iterations=10,
                 spinup_years=None,
                 save_figure=True,
                 legend_title=(r'$\bf{equilibrium}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 10'))

## plot advance figure from initial ice surface

In [ ]:
input_folder = 'plot_data/rectangular_experiments/'

plot_rec_overview(input_folder=input_folder,
                 output_folder='',
                 filename='rec_advancing_inital_sfc_overview',
                 bed_geometry='rec',
                 control_var='bed_h',
                 reg_parameter='[1., 1., 1., 10.]',
                 suffix='reg10',
                 glacier_state='adv',
                 performance_measures=['bed_h RMSE', 'sfc_h RMSE'],
                 xlim=None,
                 ylims=[[0,16],[0,10]],
                 lw = 3.,
                 ms = 15.,
                 labels_pad=-430,
                 colors=[color_1, color_2],
                 file_format='pdf',
                 dpi=300,
                 fontsize=25,
                 show_title = False,
                 line_height_multiplier=1.,
                 facecolor = 'white',
                 nr_of_iterations=10,
                 spinup_years=None,
                 save_figure=True,
                 legend_title=(r'$\bf{advancing}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 10'),
                 extra_annotation='(a)')

## plot retreating figure from inital ice surface

In [ ]:
input_folder = 'plot_data/rectangular_experiments/'

plot_rec_overview(input_folder=input_folder,
                 output_folder='',
                 filename='rec_retreat_inital_sfc_overview',
                 bed_geometry='rec',
                 control_var='bed_h',
                 reg_parameter='[1., 1., 1., 10.]',
                 suffix='reg10',
                 glacier_state='ret',
                 performance_measures=['bed_h RMSE', 'sfc_h RMSE'],
                 xlim=None,
                 ylims=[[0,16],[0,10]],
                 lw = 3.,
                 ms = 15.,
                 labels_pad=-430,
                 colors=[color_1, color_2],
                 file_format='pdf',
                 dpi=300,
                 fontsize=25,
                 show_title = False,
                 line_height_multiplier=1.,
                 facecolor = 'white',
                 nr_of_iterations=10,
                 spinup_years=None,
                 save_figure=True,
                 legend_title=(r'$\bf{retreating}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 10'),
                 extra_annotation='(b)')

## plot retreating figure from ice free surface

In [ ]:
input_folder = 'plot_data/rectangular_experiments/'

plot_rec_overview(input_folder=input_folder,
                 output_folder='',
                 filename='rec_retreat_ice_free_sfc_overview',
                 bed_geometry='rec',
                 control_var='bed_h',
                 reg_parameter='[1., 1., 1., 10.]',
                 suffix='reg10',
                 glacier_state='ret_spinup',
                 performance_measures=['bed_h RMSE', 'sfc_h RMSE'],
                 xlim=None,
                 ylims=[[0,16],[0,10]],
                 lw = 3.,
                 ms = 15.,
                 labels_pad=-430,
                 colors=[color_1, color_2],
                 file_format='pdf',
                 dpi=300,
                 fontsize=25,
                 show_title = False,
                 line_height_multiplier=1.,
                 facecolor = 'white',
                 nr_of_iterations=10,
                 spinup_years=100,
                 save_figure=True,
                  legend_title=(r'$\bf{retreating}$ Experiments starting from an $\bf{ice~free~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 10, ' +
                               r'and $t^{spinup}$ = 100 a'))

# parabolic figures

## performance plot two axis

In [ ]:
def performance_plot(axs,
                     datasets,
                     fig=None,
                     # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                     # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                     # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                     # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                     # 'widths RMSE', 'widths Diff', 'widths Bias'
                     performance_measurements=['bed_h RMSE', 'sfc_h RMSE'],
                     colors=['tab:blue', 'tab:red'],
                     xlim=[0,60],
                     line_width=2,
                     marker_size=5,
                     ylims=None,
                     nr_of_iterations=None
                    ):
    if not fig:
        fig = plt.gcf()

    all_x = []
    all_y = []

    nr_of_color = 0
    for nr_measure, measure in enumerate(performance_measurements):
        for dataset in datasets:
            if dataset is not None:
                max_index = len(dataset['computing_time'].values) - 1
                if nr_of_iterations is not None:
                    max_index = nr_of_iterations - 1
                elif xlim is not None:
                    # only consider as many points until max of xlim is reached
                    if dataset['computing_time'].values[-1] > xlim[1]:
                        max_index = np.argmax(dataset['computing_time'].values > xlim[1])

                # include time 0 for first guess
                tmp_x = [0]

                # add first guess values 
                if measure == 'bed_h RMSE':
                    tmp_y = [RMSE(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]
                elif measure == 'bed_h Diff':
                    tmp_y = [DIFF(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]
                elif measure == 'bed_h Bias':
                    tmp_y = [BIAS(dataset['first_guessed_bed_h'], dataset['true_bed_h'])]

                elif measure == 'bed_shape RMSE':
                    tmp_y = [RMSE(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]
                elif measure == 'bed_shape Diff':
                    tmp_y = [DIFF(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]
                elif measure == 'bed_shape Bias':
                    tmp_y = [BIAS(dataset['first_guessed_bed_shape'], dataset['true_bed_shape'])]

                elif measure == 'w0 RMSE':
                    tmp_y = [RMSE(dataset['first_guessed_w0'], dataset['true_w0'])]
                elif measure == 'w0 Diff':
                    tmp_y = [DIFF(dataset['first_guessed_w0'], dataset['true_w0'])]
                elif measure == 'w0 Bias':
                    tmp_y = [BIAS(dataset['first_guessed_w0'], dataset['true_w0'])]

                elif measure == 'sfc_h RMSE':
                    tmp_y = [RMSE(dataset['first_guess_surface_h'], dataset['true_surface_h'])]
                elif measure == 'sfc_h Diff':
                    tmp_y = [DIFF(dataset['first_guess_surface_h'], dataset['true_surface_h'])]
                elif measure == 'sfc_h Bias':
                    tmp_y = [DIFF(dataset['first_guess_surface_h'], dataset['true_surface_h'])]

                elif measure == 'widths RMSE':
                    tmp_y = [RMSE(dataset['first_guess_widths'], dataset['true_widths'])]
                elif measure == 'widths Diff':
                    tmp_y = [DIFF(dataset['first_guess_widths'], dataset['true_widths'])]
                elif measure == 'widths Bias':
                    tmp_y = [DIFF(dataset['first_guess_widths'], dataset['true_widths'])]

                else:
                    raise ValueError('Unknown performance measurement!')

                for i in dataset.coords['nr_of_iteration'].values[:max_index + 1] - 1:
                    tmp_x.append(dataset['computing_time'][i])
                    if measure == 'bed_h RMSE':
                        tmp_y.append(RMSE(dataset['guessed_bed_h'][i], dataset['true_bed_h']))
                        ylabel_1 = 'bed_h m'
                    elif measure == 'bed_h Diff':
                        tmp_y.append(DIFF(dataset['guessed_bed_h'][i], dataset['true_bed_h']))
                    elif measure == 'bed_h Bias':
                        tmp_y.append(BIAS(dataset['guessed_bed_h'][i], dataset['true_bed_h']))

                    elif measure == 'bed_shape RMSE':
                        tmp_y.append(RMSE(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))
                        ylabel_1 = 'bed_shape'
                    elif measure == 'bed_shape Diff':
                        tmp_y.append(DIFF(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))
                    elif measure == 'bed_shape Bias':
                        tmp_y.append(BIAS(dataset['guessed_bed_shape'][i], dataset['true_bed_shape']))

                    elif measure == 'w0 RMSE':
                        tmp_y.append(RMSE(dataset['guessed_w0'][i], dataset['true_w0']))
                        ylabel_1 = 'w0 m'
                    elif measure == 'w0 Diff':
                        tmp_y.append(DIFF(dataset['guessed_w0'][i], dataset['true_w0']))
                    elif measure == 'w0 Bias':
                        tmp_y.append(BIAS(dataset['guessed_w0'][i], dataset['true_w0']))

                    elif measure == 'sfc_h RMSE':
                        tmp_y.append(RMSE(dataset['surface_h'][i], dataset['true_surface_h']))
                        ylabel_2 = 'sfc_h m'
                    elif measure == 'sfc_h Diff':
                        tmp_y.append(DIFF(dataset['surface_h'][i], dataset['true_surface_h']))
                    elif measure == 'sfc_h Bias':
                        tmp_y.append(BIAS(dataset['surface_h'][i], dataset['true_surface_h']))

                    elif measure == 'widths RMSE':
                        tmp_y.append(RMSE(dataset['widths'][i], dataset['true_widths']))
                        ylabel_2 = 'widths m'
                    elif measure == 'widths Diff':
                        tmp_y.append(DIFF(dataset['widths'][i], dataset['true_widths']))
                    elif measure == 'widths Bias':
                        tmp_y.append(BIAS(dataset['widths'][i], dataset['true_widths']))

                    else:
                        raise ValueError('Unknown performance measurement!')
            else:
                tmp_x = []
                tmp_y = []

            axs[nr_measure].plot(tmp_x, tmp_y, '.-',
                                 color=colors[nr_of_color],
                                 lw=line_width,
                                 ms=marker_size)
            nr_of_color += 1

    axs[0].set_xlabel('time in s')
    axs[0].set_ylabel(ylabel_1, color=colors[0])
    axs[0].tick_params(axis='y', labelcolor=colors[0])
    axs[0].grid()

    if len(colors) == 2:
        axs[1].set_ylabel(ylabel_2, color=colors[1])
        axs[1].tick_params(axis='y', labelcolor=colors[1])
    elif len(colors) == 4:
        axs[1].set_ylabel(ylabel_2, color=colors[2])
        axs[1].tick_params(axis='y', labelcolor=colors[2])
    
    #for x, y, description in zip(all_x, all_y, descriptions):
    #    ax.plot(x, y, '.-', label=description)

    # ax.legend((),(),title=measure, loc='best')
    # ax.axvline(60, alpha=0.5, c='gray', ls='--')
    # ax.axvline(20, alpha=0.5, c='gray', ls='--')
    if xlim is not None:
        axs[0].set_xlim(xlim)
    
    if ylims is not None:
        axs[0].set_ylim(ylims[0])
        axs[0].set_yticks(np.linspace(ylims[0][0], ylims[0][1], 3))
        axs[1].set_ylim(ylims[1])
        axs[1].set_yticks(np.linspace(ylims[1][0], ylims[1][1], 3))
    

## Define function

In [ ]:
def plot_par_overview(input_folder,
                      output_folder='',
                      filename='par_retreat_inital_sfc_overview',
                      bed_geometry='par',
                      control_var='bed_h and bed_shape',
                      reg_parameter='[1., 1., 1., 100.]',
                      suffix='reg11',
                      glacier_state='ret',
                      performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                            ['bed_shape RMSE', 'widths RMSE']],
                      xlim=[0,60],
                      ylims=[[0,16],[0,10]],
                      lw=2,
                      ms=5,
                      labels_pad=-430,
                      colors=[color_1, color_2],
                      file_format='pdf',
                      dpi=300,
                      fontsize=25,
                      show_title = False,
                      line_height_multiplier=1.,
                      facecolor = 'white',
                      spinup_years=None,
                      nr_of_iterations=10,
                      legend_title=r'Experiments with $\lambda_0$ = 1 and $\lambda_1$ = 100',
                      save_plot=False,
                      legend_labels=[''],
                      legend_colors=[''],
                      legend_line_styles=[''],
                      legend_ncol=2):
    plt.rcParams['font.family'] = 'monospace'
    mpl.rcParams.update({'font.size': fontsize})

    # define grid for axis

    #        heading | legned
    #------------------------
    # identifier | spatial subplots | performance subplots

    # multiplier for variable plot height
    if len(input_folder) == 1:
        height_multiplier = len(suffix)
    else:
        height_multiplier = 1 # len(input_folder)

    # define fixed size of subplot
    subplot_width_spatial = 6
    subplot_width_performance = 4.5
    subplot_height = .4 + line_height_multiplier * height_multiplier
    subplot_separation_x = 2.2
    subplot_separation_y = .5
    sbuplot_minor_separation_y = .2

    # define fixed x size for identifier (first columns)
    identifier_width = 2
    identifier_separation = .05

    # define separation identifier to subplots
    separation_identifier_plots = .1

    # define hight of header line
    height_header = 2.5

    #define separation identifier/subplots to heading
    separation_heading_plots = .8

    #define separation heading to title
    separation_heading_title = .1

    # define hight of title 
    height_title = .5

    # fixed size in inch
    # along x axis                                                              x-index for locator
    horiz = [Size.Fixed(identifier_width),                                    # 0 first identifier
             Size.Fixed(identifier_separation),   
             Size.Fixed(identifier_width),                                    # 2 second identifier
             Size.Fixed(identifier_separation),   
             Size.Fixed(identifier_width),                                    # 4 third identifier
             Size.Fixed(separation_identifier_plots),
             Size.Fixed(subplot_width_spatial),                               # 6 spatial subplot column
             Size.Fixed(subplot_separation_x),
             Size.Fixed(subplot_width_performance),                           # 8 performance subplot column
            ]
                                                                                  # y-index for locator
    vert = [Size.Fixed(subplot_height),                                       # 0 3.2subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 2 3.1 subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 4 2.2 subplot 
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 6 2.1 subplot 
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 8 1.2 subplot
            Size.Fixed(subplot_separation_y),
            Size.Fixed(subplot_height),                                       # 10 1.1 subplot 
            Size.Fixed(separation_heading_plots),
            Size.Fixed(height_header),                                        # 12 header | legend 
           ]

    if show_title:
        vert += [Size.Fixed(separation_heading_title),
                 Size.Fixed(height_title)                                          # 14 title
                ]

    rect = (0., 0., 1., 1.)  # Position of the grid in the figure

    # create two figures with each 3 different geometries
    for fig_nr in range(2):
        fig = plt.figure(figsize=(1, 1), facecolor=facecolor)
        # divide the axes rectangle into grid whose size is specified by horiz * vert
        divider = Divider(fig, rect, horiz, vert, aspect=False)

        # include header line
        if bed_geometry == 'rec':
            shape_header = 'widhts'
            title_geometry = 'Rectangular'
        elif bed_geometry == 'par':
            shape_header = 'shape'
            title_geometry = 'Parabolic'
        elif bed_geometry == 'tra':
            shape_header = 'w0'
            title_geometry = 'Trapezoidal'
        else:
            raise ValueError('Unknown bed geometry!')

        if glacier_state == 'equ':
            title_glacier_state = 'equilibrium'
        elif glacier_state == 'adv':
            title_glacier_state = 'advancing'
        elif glacier_state == 'ret':
            title_glacier_state = 'retreating'
        elif glacier_state == 'ret_spinup':
            title_glacier_state = 'retreating with spinup'
        else:
            raise ValueError('Unknown glacier state')

        if control_var == 'bed_h_and_bed_shape':
            identifier_var = ['bed_h', 'bed_shape']
        elif control_var == 'bed_h_and_w0':
            identifier_var = ['bed_h', 'w0']
        else:
            raise ValueError('Unknown control variables')

        if show_title:
            # Include title
            ax = fig.subplots()
            set_text(ax, title_geometry + ' bed shape, '
                     + title_glacier_state + ' Experiments, reg_parameters = '
                     + reg_parameter,
                     facecolor=facecolor,
                     alignment='center',
                     fontsize=fontsize)
            ax.set_axes_locator(divider.new_locator(nx=0, nx1=7, ny=14))

        # include header
        for i, item in enumerate(['bed',
                                  shape_header,
                                  'shown']):
            ax = setup_axes(fig, 111)
            set_text(ax, item, facecolor=facecolor, alignment='center', fontsize=fontsize)
            ax.set_axes_locator(divider.new_locator(nx=i*2, ny=12))

        # include description of experiments
        #ax = setup_axes(fig, 111)
        #set_title_with_experiments(ax,
        #                           experiment_descriptions.copy(),
        #                           bed_geometry,
        #                           control_var,
        #                           two_control_var_options=None,
        #                           facecolor=[0.9, 0.9, 0.9],
        #                           fontsize=fontsize,
        #                           pad=10.0,
        #                           no_first_line=True)
        #ax.set_axes_locator(divider.new_locator(nx=6, ny=12))
        
        # include legend of performance plot
        ax = fig.subplots()
        add_legend(ax=ax,
               title=legend_title,
               fontsize=fontsize,
               lw=lw,
               ms=ms,
               ncol=legend_ncol,
               labels=legend_labels,
               colors=legend_colors,
               line_styles=legend_line_styles)
        #legend_plot(ax,
        #            legend_entry,
        #            colors,
        #            line_width,
        #            marker_size,
        #            legend_title = legend_title)
        ax.set_axes_locator(divider.new_locator(nx=6, nx1=9, ny=12))

        # include subplots
        bed_heights = {'line': 'linear', 'clif': 'cliff', 'rand': 'random'}
        bed_shapes = {'cons': 'constant', 'wide':'wide top'}  # 'rand': 'rand'}

        # variables to keep track of current row
        row_nr = 10

        # variable for creation of filename
        filename_bed_geometry = (bed_geometry + '_')

        count_subplots = 0
        count_geometries = 0
        # add subplots
        for bed_height in bed_heights:

            # add bed height type to filename
            filename_bed_height = (filename_bed_geometry + bed_height + '_')

            for bed_shape in bed_shapes:
                if fig_nr == 1:
                    if count_geometries < 3:
                        count_geometries += 1
                        continue

                if count_subplots > 5:
                    break
                # set first identifier 
                ax = setup_axes(fig, 111)
                set_text(ax, bed_heights[bed_height], facecolor=facecolor, alignment='center', fontsize=fontsize)
                ax.set_axes_locator(divider.new_locator(nx=0, ny=row_nr))
                # set second identifier
                ax = setup_axes(fig, 111)
                set_text(ax, bed_shapes[bed_shape], facecolor=facecolor, alignment='center', fontsize=fontsize)
                ax.set_axes_locator(divider.new_locator(nx=2, ny=row_nr))

                # add bed shape type to filename
                filename_bed_shape = (filename_bed_height + bed_shape + '_')

                # add glacier state and control var
                filename_raw = (filename_bed_shape + glacier_state + '_' + control_var)
                
                datasets = []
                for file_suffix in suffix:
                    if spinup_years is None:
                        # add suffix and ending to filename
                        filename_nc = filename_raw + '_' + file_suffix + '.nc'
                    else:
                        filename_nc = filename_raw + '_' + str(spinup_years) + '_' + file_suffix + '.nc'

                    # get datsets
                    if filename_nc in os.listdir(input_folder):
                        with xr.open_dataset(input_folder + filename_nc) as ds:
                            datasets.append(ds)
                    else:
                        datasets.append(None)

                for i_control_var, singel_control_var in enumerate(identifier_var):
                    # include third identifier
                    ax = setup_axes(fig, 111)
                    if singel_control_var == 'bed_shape':
                        plot_control_var = r'Ps'
                    else:
                        plot_control_var = singel_control_var
                    set_text(ax, plot_control_var, facecolor=facecolor, alignment='center', fontsize=fontsize)
                    ax.set_axes_locator(divider.new_locator(nx=4, ny=row_nr))
                    
                    # define colormap
                    if singel_control_var == 'bed_h':
                        cmap='Spectral'
                    else:
                        cmap='vlag'
                    
                    # plot spatial distribution
                    ax = setup_axes(fig, 111)
                    im = heatmap(datasets,
                                 opti_var=singel_control_var,
                                 fig=fig,
                                 ax=ax,
                                 cmap=cmap,
                                 grid_color=facecolor,
                                 presentation=True,
                                 labels_pad=labels_pad,
                                 xlim=xlim,
                                 nr_of_iterations=nr_of_iterations)
                    ax.set_axes_locator(divider.new_locator(nx=6, ny=row_nr))

                    # plot performance plot
                    ax = fig.subplots()
                    ax.set_axes_locator(divider.new_locator(nx=8, ny=row_nr))
                    #ax2 = ax.twinx()
                    #ax2.set_axes_locator(divider.new_locator(nx=8, ny=row_nr))
                    if performance_measures[i_control_var][0] == 'bed_shape RMSE':
                        y_label = ''
                    elif performance_measures[i_control_var] == 'w0 RMSE':
                        y_label = 'm'
                    else:
                        y_label = 'm'
                    #y_labels=['RMSE (m)', r'$Ps$']
                    performance_plot_one_y_axis(ax,
                                                 datasets,
                                                 fig=None,
                                                 # 'bed_h RMSE', 'bed_h Diff', 'bed_h Bias',
                                                 # 'bed_shape RMSE', 'bed_shape Diff', 'bed_shape Bias',
                                                 # 'w0 RMSE', 'w0 Diff', 'w0 Bias',
                                                 # 'sfc_h RMSE', 'sfc_h Diff', 'sfc_h Bias',
                                                 # 'widths RMSE', 'widths Diff', 'widths Bias'
                                                 performance_measurement=performance_measures[i_control_var],
                                                 colors=colors,
                                                 xlim=None,
                                                 y_label=y_label, #y_labels[i_control_var],
                                                 annotation=None,
                                                 annotation_x_position=-0.2,
                                                 annotation_y_position=1,
                                                 lw=lw,
                                                 fontsize=fontsize,
                                                 ms=ms,
                                                 nr_of_iterations=nr_of_iterations
                                                )
                    #performance_plot([ax, ax2],
                    #                 datasets,
                    #                 fig=fig,
                    #                 xlim=xlim,
                    #                 line_width=line_width,
                    #                 marker_size=marker_size,
                    #                 performance_measurements=performance_measures[i_control_var],
                    #                 colors=colors,
                    #                 ylims=ylims[i_control_var],
                    #                 nr_of_iterations=nr_of_iterations
                    #                )

                    count_subplots += 1
                    row_nr -= 2
        
        if save_plot:
            fig.savefig(output_folder + filename + '_' + str(fig_nr) + '.' + file_format,
                        format=file_format,
                        bbox_inches='tight',
                        dpi=dpi)
        

## plot equilibrium figure from initial ice surface

In [ ]:
input_folder = 'plot_data/parabolic_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='par_equilibrium_inital_sfc_overview',
                  bed_geometry='par',
                  control_var='bed_h_and_bed_shape',
                  reg_parameter='[1., 1., 1., 100.]',
                  suffix=['at_once_scal1e4reg11',
                          'calculatedreg11',
                          'separatedreg11'],
                  glacier_state='equ',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['bed_shape RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{equilibrium}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 100'),
                  #r'Experiments with $\lambda_0$ = 1 and $\lambda_1$ = 100',
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $Ps$ and $Ps_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

## plot advance figure from initial ice surface

In [ ]:
input_folder = 'plot_data/parabolic_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='par_advancing_inital_sfc_overview',
                  bed_geometry='par',
                  control_var='bed_h_and_bed_shape',
                  reg_parameter='[1., 1., 1., 100.]',
                  suffix=['at_once_scal1e4reg11',
                          'calculatedreg11',
                          'separatedreg11'],
                  glacier_state='adv',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['bed_shape RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{advancing}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 100'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $Ps$ and $Ps_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

## plot retreating figure from initial ice surface

In [ ]:
input_folder = 'plot_data/parabolic_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='par_retreat_inital_sfc_overview',
                  bed_geometry='par',
                  control_var='bed_h_and_bed_shape',
                  reg_parameter='[1., 1., 1., 100.]',
                  suffix=['at_once_scal1e4reg11',
                          'calculatedreg11',
                          'separatedreg11'],
                  glacier_state='ret',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['bed_shape RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{retreating}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 100'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 'and A: at once approach',
                                 'and B: calculated approach',
                                 'and C: separated approach',
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $Ps$ and $Ps_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

## plot retreating figure from ice free surface

In [ ]:
input_folder = 'plot_data/parabolic_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='par_retreat_ice_free_sfc_overview',
                  bed_geometry='par',
                  control_var='bed_h_and_bed_shape',
                  reg_parameter='[1., 1., 1., 100.]',
                  suffix=['at_once_scal1e4_100reg11',
                          'calculated_100reg11',
                          'separated_100reg11'],
                  glacier_state='ret_spinup',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['bed_shape RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{retreating}$ Experiments starting from an $\bf{ice~free~ glacier~ surface}$,' +
                               '\n' + 
                               r'regularisation parameters $\lambda_0$ = 1 and $\lambda_1$ = 100, ' +
                               r'$t^{spinup}$ = 100 a'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $Ps$ and $Ps_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

# trapezoidal figures

## plot equilibrium figure from initial ice surface

In [ ]:
input_folder = 'plot_data/trapezoidal_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='tra_equilibrium_inital_sfc_overview',
                  bed_geometry='tra',
                  control_var='bed_h_and_w0',
                  reg_parameter='[1., 1., 10., 100.]',
                  suffix=['at_oncereg13',
                          'calculatedreg13',
                          'separatedreg13'],
                  glacier_state='equ',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['w0 RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{equilibrium}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 10 and $\lambda_1$ = 100'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $w0$ and $w0_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

## plot advance figure from initial ice surface

In [ ]:
input_folder = 'plot_data/trapezoidal_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='tra_advancing_inital_sfc_overview',
                  bed_geometry='tra',
                  control_var='bed_h_and_w0',
                  reg_parameter='[1., 1., 10., 100.]',
                  suffix=['at_oncereg13',
                          'calculatedreg13',
                          'separatedreg13'],
                  glacier_state='adv',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['w0 RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{advancing}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 10 and $\lambda_1$ = 100'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $w0$ and $w0_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

## plot retreating figure from initial ice surface

In [ ]:
input_folder = 'plot_data/trapezoidal_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='tra_retreating_inital_sfc_overview',
                  bed_geometry='tra',
                  control_var='bed_h_and_w0',
                  reg_parameter='[1., 1., 10., 100.]',
                  suffix=['at_oncereg13',
                          'calculatedreg13',
                          'separatedreg13'],
                  glacier_state='ret',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['w0 RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{retreating}$ Experiments starting from an $\bf{initial~ glacier~ surface}$,' +
                               '\n' + 
                               r'with regularisation parameters $\lambda_0$ = 10 and $\lambda_1$ = 100'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $w0$ and $w0_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)

## plot retreating figure from ice free surface

In [ ]:
input_folder = 'plot_data/trapezoidal_experiments/'

plot_par_overview(input_folder=input_folder,
                  output_folder='',
                  filename='tra_retreating_ice_free_sfc_overview',
                  bed_geometry='tra',
                  control_var='bed_h_and_w0',
                  reg_parameter='[1., 1., 10., 100.]',
                  suffix=['at_once_100reg13',
                          'calculated_100reg13',
                          'separated_100reg13'],
                  glacier_state='ret_spinup',
                  performance_measures=[['bed_h RMSE', 'sfc_h RMSE'],
                                        ['w0 RMSE']],
                  xlim=[0,60],
                  ylims=[[0,16],[0,10]],
                  lw=3,
                  ms=15,
                  labels_pad=-430,
                  colors=[color_1, color_2, color_3],
                  file_format='pdf',
                  dpi=300,
                  fontsize=25,
                  show_title = False,
                  line_height_multiplier=2.,
                  facecolor = 'white',
                  spinup_years=None,
                  nr_of_iterations=10,
                  legend_title=(r'$\bf{retreating}$ Experiments starting from an $\bf{ice~free~ glacier~ surface}$,' +
                               '\n' + 
                               r'regularisation parameters $\lambda_0$ = 10 and $\lambda_1$ = 100, ' +
                               r'$t^{spinup}$ = 100 a'),
                  save_plot=True,
                  legend_labels=['fg: first guess',
                                 "and A: 'explicit' approach",
                                 "and B: 'implicit' approach",
                                 "and C: 'iterative' approach",
                                 r'RMSE of $b$ and $b_{t}$',
                                 r'RMSE of $s^{e}_m$ and $s^{e}_o$',
                                 r'RMSE of $w0$ and $w0_{t}$'],
                  legend_colors=['none',
                                 color_1,
                                 color_2,
                                 color_3,
                                 axis_color,
                                 axis_color,
                                 axis_color],
                  legend_line_styles=['',
                                      '-',
                                      '-',
                                      '-',
                                      '.-',
                                      '.--',
                                      '.:'],
                  legend_ncol=2)